In [1]:
import numpy as np
import tensorflow as tf
from datetime import datetime
from keras.models import Sequential, load_model
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.utils.training_utils import multi_gpu_model
from tqdm import tqdm

/home/derek/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/derek/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# read in data
cardtext = [list(x) for x in list(np.load('data/card_texts_new.npy'))]
c2i = np.load('data/c2i.npy').item()
i2c = np.load('data/i2c.npy').item()

In [3]:
# test - randomize!
np.random.seed = 1337
indices = list(np.random.permutation(len(cardtext)))
cardtext = [cardtext[i] for i in indices]
# cardtext = cardtext

In [4]:
# set parameters

DROP_RATE = 0.2 # dropout
EMBEDDING_SIZE = 128 # embedding size
HIDDEN_SIZE = 192 # lstm feature vector
HIDDEN_LAYERS = 3 # number of layers
START_EPOCH = 0
VOCAB_SIZE = len(c2i.keys()) # number of characters

WINDOW_SIZE = 100 # context length
BATCH_SIZE = 64
CARDS_PER_BATCH = 5
NUM_EPOCHS = 1000

OUT_INCREMENT = 100 # printout after n batches - and save

In [5]:
def cardGenerator(cardtext, windowsize, cards_per_batch, c2i=c2i, debug=False):
    
    i = 0
    indices = list(np.random.permutation(len(cardtext)))
    idx = indices[i]
    lastseq = ''
    
    def nextcard(cardtext, idx, debug=debug):
        if debug:
            card_idx = cardtext[idx]
        else:
            card_idx = [c2i[c] for c in cardtext[idx]]
        
        return list(card_idx)
    
    # pregenerate warmup sequence of windowsize
    # get startup sequence with card longer than windowsize+1
    while len(cardtext[idx]) < windowsize:
        i += 1
        idx = indices[i]
    # 'pad' sequence with end of card
    if debug:
        sequence = list(cardtext[idx][-(windowsize):])
    else:
        sequence = list([c2i[c] for c in cardtext[idx][-(windowsize):]])
    i += 1
    idx = indices[i]
    # add n cards to sequence where n = cards_per_batch
    for j in range(cards_per_batch):
        sequence += nextcard(cardtext, idx)
        i += 1
        idx = indices[i]
        
    # create matrix
    x = []
    y = []
    
    # main iterator
    while True:
        
        # set lastseq for next cycle
        laststr = sequence[-(windowsize):]

        # generate batch (of cards_per_batch cards)
        while len(sequence) > windowsize:
            x.append(np.array(sequence[:windowsize]))
            y.append(sequence[windowsize])
            sequence.pop(0)
        
        # generate window-shifted data
        # reshape for sparse_categorical_crossentropy
        sequence = []
        y = np.array(y)
        y = y[:, np.newaxis]
        # yield and reset
        yield(np.asarray(x), y)
        x, y = [], []
        
        # for next batch, check if enough remaining, else reset
        if len(indices[i:]) < cards_per_batch:
            indices = np.random.permutation(len(cardtext))
            i = 0
            idx = indices[i]
        else:
            i += 1
            idx = indices[i]
            
        # seed warmup sequence with end of last batch
        sequence = list(laststr)

        i += 1
        idx = indices[i]
        for j in range(cards_per_batch):
            sequence += nextcard(cardtext, idx)
            i += 1
            idx = indices[i]

In [6]:
getbatch = cardGenerator(cardtext, WINDOW_SIZE, CARDS_PER_BATCH)

In [7]:
# define model on CPU
with tf.device("/cpu:0"):
    model = Sequential()
    model.add(Embedding(VOCAB_SIZE, EMBEDDING_SIZE, 
                        input_shape=(WINDOW_SIZE, )))
    model.add(Dropout(DROP_RATE))
    for _ in range(HIDDEN_LAYERS-1):
        model.add(LSTM(HIDDEN_SIZE, return_sequences=True))
    model.add(LSTM(HIDDEN_SIZE))
    model.add(Dense(VOCAB_SIZE, activation='softmax'))

# make multi-gpu model with 2 GPU's
model = multi_gpu_model(model, gpus=2)

In [8]:
# compile
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

In [9]:
# predict 'Ⓔ'

def predict(startchars='random', temperature=0.5, maxlen=800):
    
    seq_out = []
    
    if temperature=='random':
        tmp = np.random.random()
    else:
        tmp = temperature
    
    # starting sequence
    if startchars=='none':
        seq_in = [c2i['Ⓔ'] for i in range(WINDOW_SIZE)]
    
    elif startchars=='random':
        seq_in = []
        alpha = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k',
                 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v',
                 'w', 'x', 'y', 'z']
        alpha = [a for a in alpha if a in c2i.keys()]
        while len(seq_in) < WINDOW_SIZE-1:
            rnd = np.random.randint(0, len(alpha))
            seq_in += [c2i[alpha[rnd]]]
        seq_in += [c2i['Ⓔ']]
    
    else:
        s = list(startchars)
        s = s[:WINDOW_SIZE]
        seq_out =  [c2i[c] for c in s]
        while len(s) < WINDOW_SIZE:
            s.insert(0, 'Ⓔ')
        seq_in = [c2i[c] for c in s]
        
    # softmax temperature
    # scaling factor of logits = logits/temperature
    # high temp = more confident = more diverse, more mistakes
    # low temp: more conservative
    # https://stackoverflow.com/questions/37246030/how-to-change-the-temperature-of-a-softmax-output-in-keras/37254117#37254117
    def sample(a, temperature=tmp):
        a = np.array(a)**(1/temperature)
        p_sum = a.sum()
        sample_temp = a/p_sum 

        # stupid fix for > 1 error
        while sum(sample_temp) > 1:
            sample_temp[0] -= 0.0001

        return np.argmax(np.random.multinomial(1, sample_temp, 1))

    for i in range(maxlen):

        # predict next char
        pred_out = model.predict(np.array(seq_in).reshape((1, WINDOW_SIZE)))
        # get index of highest pred
        idx = sample(pred_out[0])
        # save index for decoding
        seq_out.append(idx)
        # add index to input sequence
        seq_in.append(int(idx))
        # remove earliest
        seq_in.pop(0)

    # decode final sequence
    card_char = ''.join([i2c[int(i)] for i in seq_out])
    card_char = card_char.replace('·', '|')
    card_char = card_char.replace('Ⓔ', 'Ⓔ\n|')
    card_text = card_char.split('|')

    for f in card_text:
        print(f)
        
    return card_text

In [ ]:
# load model
model.load_weights('model/v4-multi2-modelweights-epoch6-cards5000-2018-02-05 17:27:15.520281.h5')
START_EPOCH = 6

In [ ]:
# just train, fix epochs later
for epoch_idx in range(START_EPOCH, NUM_EPOCHS):
    # epochs are "meaningless" here, roughly batch size = n cards so...
    for batch in range(int(len(cardtext)/CARDS_PER_BATCH)):
        print('epoch', epoch_idx+1, 'batch',batch+1, str(datetime.now()))
        # get batch
        x_batch, y_batch = next(getbatch)
        
        # fit to card batch
        r = model.fit(x_batch, y_batch, batch_size=BATCH_SIZE, epochs=1, verbose=0)

        if batch % OUT_INCREMENT == 0 and batch > 0:
            model.save_weights('model/v4-multi2-modelweights-epoch{}-cards{}-{}.h5'.format(epoch_idx+1, CARDS_PER_BATCH*(batch), str(datetime.now())))
            # change to writing to file as well as printing
            a = predict(startchars='random', temperature='random')
            b = predict(startchars='random', temperature='random')
            c = predict(startchars='random', temperature='random')
            filename = 'sample/v4-multi2-cardsamples-epoch{}-cards{}-{}.txt'.format(epoch_idx+1, CARDS_PER_BATCH*(batch), str(datetime.now()))
            print("\nwriting file", filename)
            with open(filename, 'w') as f:
                for prd in [a, b, c]:
                    for ln in prd:
                        f.write(ln)
                        f.write('\n')
                f.write('\n')
            
            print('***************************************************************')

epoch 7 batch 1 2018-02-05 19:27:12.710068
epoch 7 batch 2 2018-02-05 19:27:20.174926
epoch 7 batch 3 2018-02-05 19:27:26.792809
epoch 7 batch 4 2018-02-05 19:27:30.272326
epoch 7 batch 5 2018-02-05 19:27:35.306334
epoch 7 batch 6 2018-02-05 19:27:39.697476
epoch 7 batch 7 2018-02-05 19:27:44.956613
epoch 7 batch 8 2018-02-05 19:27:49.305318
epoch 7 batch 9 2018-02-05 19:27:52.763251
epoch 7 batch 10 2018-02-05 19:27:57.119972
epoch 7 batch 11 2018-02-05 19:28:03.175141
epoch 7 batch 12 2018-02-05 19:28:07.120366
epoch 7 batch 13 2018-02-05 19:28:12.003639
epoch 7 batch 14 2018-02-05 19:28:15.411895
epoch 7 batch 15 2018-02-05 19:28:21.427823
epoch 7 batch 16 2018-02-05 19:28:25.227622
epoch 7 batch 17 2018-02-05 19:28:29.624724
epoch 7 batch 18 2018-02-05 19:28:33.473288
epoch 7 batch 19 2018-02-05 19:28:36.466883
epoch 7 batch 20 2018-02-05 19:28:40.780783
epoch 7 batch 21 2018-02-05 19:28:43.778488
epoch 7 batch 22 2018-02-05 19:28:48.619362
epoch 7 batch 23 2018-02-05 19:28:52.0859

epoch 7 batch 107 2018-02-05 19:38:26.741086
epoch 7 batch 108 2018-02-05 19:38:32.392295
epoch 7 batch 109 2018-02-05 19:38:37.228420
epoch 7 batch 110 2018-02-05 19:38:41.618560
epoch 7 batch 111 2018-02-05 19:38:45.060065
epoch 7 batch 112 2018-02-05 19:38:49.507506
epoch 7 batch 113 2018-02-05 19:38:53.856602
epoch 7 batch 114 2018-02-05 19:38:57.799767
epoch 7 batch 115 2018-02-05 19:39:01.813826
epoch 7 batch 116 2018-02-05 19:39:06.181612
epoch 7 batch 117 2018-02-05 19:39:11.075483
epoch 7 batch 118 2018-02-05 19:39:15.880793
epoch 7 batch 119 2018-02-05 19:39:20.750666
epoch 7 batch 120 2018-02-05 19:39:24.693500
epoch 7 batch 121 2018-02-05 19:39:28.205829
epoch 7 batch 122 2018-02-05 19:39:32.139762
epoch 7 batch 123 2018-02-05 19:39:37.380094
epoch 7 batch 124 2018-02-05 19:39:42.327447
epoch 7 batch 125 2018-02-05 19:39:47.630101
epoch 7 batch 126 2018-02-05 19:39:51.556082
epoch 7 batch 127 2018-02-05 19:39:57.278100
epoch 7 batch 128 2018-02-05 19:40:02.479282
epoch 7 ba

epoch 7 batch 232 2018-02-05 19:51:00.485147
epoch 7 batch 233 2018-02-05 19:51:05.336885
epoch 7 batch 234 2018-02-05 19:51:10.574646
epoch 7 batch 235 2018-02-05 19:51:15.851650
epoch 7 batch 236 2018-02-05 19:51:21.535484
epoch 7 batch 237 2018-02-05 19:51:25.447153
epoch 7 batch 238 2018-02-05 19:51:29.786785
epoch 7 batch 239 2018-02-05 19:51:35.919162
epoch 7 batch 240 2018-02-05 19:51:40.294185
epoch 7 batch 241 2018-02-05 19:51:47.348699
epoch 7 batch 242 2018-02-05 19:51:51.778556
epoch 7 batch 243 2018-02-05 19:51:57.012394
epoch 7 batch 244 2018-02-05 19:52:00.074243
epoch 7 batch 245 2018-02-05 19:52:03.987569
epoch 7 batch 246 2018-02-05 19:52:09.181153
epoch 7 batch 247 2018-02-05 19:52:14.849067
epoch 7 batch 248 2018-02-05 19:52:19.611825
epoch 7 batch 249 2018-02-05 19:52:24.857890
epoch 7 batch 250 2018-02-05 19:52:28.817217
epoch 7 batch 251 2018-02-05 19:52:33.231148
epoch 7 batch 252 2018-02-05 19:52:38.440351
epoch 7 batch 253 2018-02-05 19:52:42.378542
epoch 7 ba

/home/derek/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in true_divide


                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
elvish
ⓍⓌ
C
creature
human
soldier
2
2
whenever Ⓝ becomes blocked, you may return target creature card from your graveyard to your hand.Ⓔ

sand savage
①Ⓦ
U
enchantment
aura
enchant creature
enchanted

...

epoch 8 batch 2747 2018-02-06 10:59:53.361155
epoch 8 batch 2748 2018-02-06 11:00:00.719100
epoch 8 batch 2749 2018-02-06 11:00:03.325955
epoch 8 batch 2750 2018-02-06 11:00:06.734961
epoch 8 batch 2751 2018-02-06 11:00:11.456722
epoch 8 batch 2752 2018-02-06 11:00:15.383682
epoch 8 batch 2753 2018-02-06 11:00:20.598459
epoch 8 batch 2754 2018-02-06 11:00:26.256597
epoch 8 batch 2755 2018-02-06 11:00:31.054693
epoch 8 batch 2756 2018-02-06 11:00:36.339862
epoch 8 batch 2757 2018-02-06 11:00:40.696481
epoch 8 batch 2758 2018-02-06 11:00:44.580625
epoch 8 batch 2759 2018-02-06 11:00:50.765541
epoch 8 batch 2760 2018-02-06 11:00:54.709014
epoch 8 batch 2761 2018-02-06 11:00:58.591390
epoch 8 batch 2762 2018-02-06 11:01:04.276950
epoch 8 batch 2763 2018-02-06 11:01:08.705954
epoch 8 batch 2764 2018-02-06 11:01:13.105624
epoch 8 batch 2765 2018-02-06 11:01:18.359505
epoch 8 batch 2766 2018-02-06 11:01:21.813610
epoch 8 batch 2767 2018-02-06 11:01:24.877683
epoch 8 batch 2768 2018-02-06 11:0

epoch 8 batch 2869 2018-02-06 11:12:25.801714
epoch 8 batch 2870 2018-02-06 11:12:31.119514
epoch 8 batch 2871 2018-02-06 11:12:35.115166
epoch 8 batch 2872 2018-02-06 11:12:38.228878
epoch 8 batch 2873 2018-02-06 11:12:42.558522
epoch 8 batch 2874 2018-02-06 11:12:49.118673
epoch 8 batch 2875 2018-02-06 11:12:54.438601
epoch 8 batch 2876 2018-02-06 11:12:57.496316
epoch 8 batch 2877 2018-02-06 11:13:00.521383
epoch 8 batch 2878 2018-02-06 11:13:05.727711
epoch 8 batch 2879 2018-02-06 11:13:12.297930
epoch 8 batch 2880 2018-02-06 11:13:16.685623
epoch 8 batch 2881 2018-02-06 11:13:20.944898
epoch 8 batch 2882 2018-02-06 11:13:24.420345
epoch 8 batch 2883 2018-02-06 11:13:28.405389
epoch 8 batch 2884 2018-02-06 11:13:31.413102
epoch 8 batch 2885 2018-02-06 11:13:34.502729
epoch 8 batch 2886 2018-02-06 11:13:38.389789
epoch 8 batch 2887 2018-02-06 11:13:42.340198
epoch 8 batch 2888 2018-02-06 11:13:47.974361
epoch 8 batch 2889 2018-02-06 11:13:55.917575
epoch 8 batch 2890 2018-02-06 11:1

epoch 8 batch 2991 2018-02-06 11:25:00.391054
epoch 8 batch 2992 2018-02-06 11:25:03.063292
epoch 8 batch 2993 2018-02-06 11:25:08.250145
epoch 8 batch 2994 2018-02-06 11:25:13.995936
epoch 8 batch 2995 2018-02-06 11:25:17.936988
epoch 8 batch 2996 2018-02-06 11:25:23.184627
epoch 8 batch 2997 2018-02-06 11:25:27.155800
epoch 8 batch 2998 2018-02-06 11:25:32.402584
epoch 8 batch 2999 2018-02-06 11:25:35.916724
epoch 8 batch 3000 2018-02-06 11:25:40.288345
epoch 8 batch 3001 2018-02-06 11:25:46.818211
exe
①Ⓦ
U
enchantment
aura
enchant creature
enchanted creature gets +1/+1.
whenever equipped creature has protection from blue.
remove a black creature token.Ⓔ

untaker of the spirit
③ⒷⒷ
R
sorcery
reveal the top card of your library. if you do, Ⓝ deals 1 damage to target creature or player.Ⓔ

surkearer
②Ⓤ
C
creature
human
cleric
1
1
↷: target creature you control gains 3 life.Ⓔ

stassal servant
③Ⓡ
C
creature
human
soldier
1
1
flying
when Ⓝ dies, exile Ⓝ.Ⓔ

serra asland
①Ⓡ
U
instant
counderk

chant
②Ⓡ
C
sorcery
return target creature card from your graveyard to your hand.Ⓔ

stinger of the angelic
②Ⓤ
C
creature
human
wizard
2
1
when Ⓝ enters the battlefield, you may pay ①. if you do, you may search your library for a basic land cards.Ⓔ

stall of the angel
②ⓇⓇ
R
creature
human
wizard
2
2
when Ⓝ enters the battlefield, you may put that card onto the battlefield under your control to its owner's graveyard.Ⓔ

strand of the treefolk
②Ⓑ
C
creature
human
wizard
1
1
↷: target creature gets +1/+1 and gains trample until end of turn.Ⓔ

consume of the angel
②ⒷⒷ
R
creature
human
wizard
2
2
when Ⓝ enters the battlefield, you may pay ①. if you do, you may pay ①. if you do, you may pay ①. if you do, shuffle your library.Ⓔ

sundermith guardian
③
U
artifact
equipment
equipped creature gets +1/+1 and has f
monderphiny
①Ⓤ
U
instant
search your library for a basic land to its owner's hand.Ⓔ

wood of the combator
②Ⓖ
U
creature
bird
1
1
flying
when Ⓝ enters the battlefield, you may gain 2 life.Ⓔ


epoch 8 batch 3203 2018-02-06 11:50:10.509003
epoch 8 batch 3204 2018-02-06 11:50:13.577292
epoch 8 batch 3205 2018-02-06 11:50:17.945128
epoch 8 batch 3206 2018-02-06 11:50:24.114145
epoch 8 batch 3207 2018-02-06 11:50:32.073099
epoch 8 batch 3208 2018-02-06 11:50:37.306170
epoch 8 batch 3209 2018-02-06 11:50:42.070582
epoch 8 batch 3210 2018-02-06 11:50:46.879077
epoch 8 batch 3211 2018-02-06 11:50:51.228649
epoch 8 batch 3212 2018-02-06 11:50:56.012168
epoch 8 batch 3213 2018-02-06 11:51:00.778126
epoch 8 batch 3214 2018-02-06 11:51:05.548020
epoch 8 batch 3215 2018-02-06 11:51:10.715355
epoch 8 batch 3216 2018-02-06 11:51:15.011220
epoch 8 batch 3217 2018-02-06 11:51:18.979876
epoch 8 batch 3218 2018-02-06 11:51:23.886334
epoch 8 batch 3219 2018-02-06 11:51:28.204862
epoch 8 batch 3220 2018-02-06 11:51:31.756833
epoch 8 batch 3221 2018-02-06 11:51:35.637023
epoch 8 batch 3222 2018-02-06 11:51:40.477774
epoch 8 batch 3223 2018-02-06 11:51:44.845760
epoch 8 batch 3224 2018-02-06 11:5

In [ ]:
# # todo: just save to json one time
# model.save('model/100test_model.h5')
# print("saved model to disk\n")
# model.save_weights('model/100test_model_weights.h5')
# print("saved model weights to disk\n")

In [ ]:
# # load model
# model.load_weights('model/100-modelweights-epoch71-batch99.h5')
# START_EPOCH = 25

In [ ]:
predict(startchars='random', temperature='random')